# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps  #pip install python-gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Asău,46.43,26.40,46.40,81,100,6.93,RO,1603328317
1,1,Talcahuano,-36.72,-73.12,55.40,76,0,13.87,CL,1603328317
2,2,Castro,-24.79,-50.01,59.67,94,19,8.55,BR,1603328317
3,3,Mataura,-46.19,168.86,64.99,51,86,4.00,NZ,1603328317
4,4,Thinadhoo,0.53,72.93,82.74,76,71,15.03,MV,1603328317
...,...,...,...,...,...,...,...,...,...,...
565,565,Charter Township of Clinton,42.59,-82.92,53.60,100,40,4.70,US,1603328213
566,566,Richards Bay,-28.78,32.04,62.85,85,2,5.93,ZA,1603328384
567,567,Ternate,0.80,127.40,83.16,74,100,5.79,ID,1603328384
568,568,Marechal Floriano,-20.41,-40.68,75.20,88,90,2.24,BR,1603328384


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) #Configure gmaps 
locations = weather_data[["Latitude","Longitude"]] #grab lat and lng
humidity = weather_data["Humidity"] #grab humidity levels for weight

In [4]:
#Add Heatmap layer to map
#https://jupyter-gmaps.readthedocs.io/en/latest/api.html
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heatmap)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

This is a heat map of the 570 cities radomly selected from the open weathermap API. Red is max intensity of the heat.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
ideal_weather = weather_data[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80) &\
                                (weather_data["Wind Speed"] < 10) & (weather_data["Cloudiness"] == 0)].dropna()
ideal_weather

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,Makkah al Mukarramah,21.43,39.83,73.33,23,0,1.74,SA,1603328034
103,103,Maragogi,-9.01,-35.22,76.15,88,0,7.81,BR,1603328184
136,136,Airai,-8.93,125.41,78.62,43,0,1.10,TL,1603328331
162,162,Ruteng,-8.61,120.47,75.69,68,0,0.92,ID,1603328334
210,210,Parkes,-33.13,148.18,75.99,57,0,1.99,AU,1603328339
230,230,Assiut,27.18,31.18,75.20,47,0,9.17,EG,1603328341
246,246,Ratangarh,28.08,74.60,72.12,43,0,6.82,IN,1603328343
274,274,Morondava,-20.28,44.28,71.83,88,0,9.71,MG,1603328346
308,308,Cayenne,4.93,-52.33,78.80,94,0,9.08,GF,1603328351
413,413,Ceyhan,37.02,35.82,75.00,88,0,5.82,TR,1603328364


Above I'm looking for the ideal weather to meet the criteria spelled out in the instructions; 
-max temperature lower than 80 degrees but higher than 70
-Wind speed less than 10 mph
-Zero cloudiness
and dropped any rows that didn't contain all three conditions above. 
The results are ideal weather conditions.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#created df
hotel_df = ideal_weather.loc[:,["City", "Latitude", "Longitude", "Country"]]
#create column for hotel name
hotel_df["Hotel Name"] = ""

In [7]:
#found syntax for the commands at the sites indicated below
#https://developers.google.com/places/web-service/search  <<= lists parameters and search result types returned
#https://developers.google.com/places/web-service/supported_types#table1 <<= use this for Place Types
#https://softauthor.com/google-maps-places-api-nearby-search-request/ <<= shows how to use query parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#constructing parameters to conduct the search
params = {"key" : g_key, "type" : "hotel", "keyword" : "hotel", "radius" : 5000}

In [8]:
#loop through list of cities to find nearest hotel
for index, row in hotel_df.iterrows():
    # get city name, latitude, and longitude from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{latitude},{longitude}"

    # assemble url and make API request
    print(f"Requesting results for Index {index}: {city_name}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # process retrieved info if returned results are valid, 
    # error trap because I found a city that had generated an IndexError
    try:
        print(f"First hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except:
        print(f"Skipping: {city_name}")

Requesting results for Index 39: Makkah al Mukarramah
First hotel in Makkah al Mukarramah is Pullman Zamzam Makkah.
Requesting results for Index 103: Maragogi
First hotel in Maragogi is Salinas do Maragogi All Inclusive Resort.
Requesting results for Index 136: Airai
Skipping: Airai
Requesting results for Index 162: Ruteng
First hotel in Ruteng is Sky Flores Hotel.
Requesting results for Index 210: Parkes
First hotel in Parkes is Hotel Gracelands.
Requesting results for Index 230: Assiut
First hotel in Assiut is Florence Hotel Assiut.
Requesting results for Index 246: Ratangarh
First hotel in Ratangarh is Parma hotel.
Requesting results for Index 274: Morondava
First hotel in Morondava is Palissandre Cote Ouest resort & SPA.
Requesting results for Index 308: Cayenne
First hotel in Cayenne is Royal Amazonia.
Requesting results for Index 413: Ceyhan
First hotel in Ceyhan is Otel Emre.
Requesting results for Index 418: Breves
First hotel in Breves is Hotel Paraiso Bom Jesus Breves.
Reques

The above is looping through cities to find the nearest hotel.  Populating the "Hotel Name" data with results of the data request.

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
# https://jupyter-gmaps.readthedocs.io/en/latest/api.html
marker_layer = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

This is a map of the preferred cities with the nearest hotels indicated.  *The information used to disply when you hovered over them but for some reason they are not doing it any longer. 